#Load Data

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kuliah/TKC/Code"

In [ ]:
%cd /content/drive/MyDrive/Kuliah/TKC/Code

In [ ]:
ls

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

In [ ]:
#1.1 Unzip data
from zipfile import ZipFile
file_name = "face-mask-12k-images-dataset.zip"

with ZipFile(file_name,'r') as zip :
  zip.extractall()
  print('Berhasil')

In [ ]:
#Define a path
base_dir = '/content/drive/MyDrive/Kuliah/TKC/Code/Face Mask Dataset'
train_dir = os.path.join(base_dir, 'Train')
validation_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Test')


train_with_mask_dir = os.path.join(train_dir, 'WithMask')
train_without_mask_dir = os.path.join(train_dir, 'WithoutMask')

validation_with_mask_dir = os.path.join(validation_dir, 'WithMask')
validation_without_mask_dir = os.path.join(validation_dir, 'WithoutMask')

test_with_mask_dir = os.path.join(test_dir, 'WithMask')
test_without_mask_dir = os.path.join(test_dir, 'WithoutMask')

In [ ]:
print('Train : ')

print('Train with_mask : ', len(os.listdir(train_with_mask_dir)))
print('Train without_mask : ', len(os.listdir(train_without_mask_dir)))
print(""*2)
print('Validation : ')

print('Validationn with_mask : ', len(os.listdir(validation_with_mask_dir)))
print('Validation without_mask : ', len(os.listdir(validation_without_mask_dir)))
print(""*2)
print('Test : ')

print('Test with_mask : ', len(os.listdir(test_with_mask_dir)))
print('Test without_mask : ', len(os.listdir(test_without_mask_dir)))

#Preprocessing

In [ ]:
height = 100
width = 100
batch_size = 32

In [ ]:
!pip3 install keras

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Kuliah/TKC/Code/Face Mask Dataset/Train"
VALIDATION_DIR = "/content/drive/MyDrive/Kuliah/TKC/Code/Face Mask Dataset/Validation"
TEST_DIR = "/content/drive/MyDrive/Kuliah/TKC/Code/Face Mask Dataset/Test"

In [ ]:
generator_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_gen = ImageDataGenerator(rescale = 1./255)

train_generator = generator_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(height, width),
    class_mode='categorical',
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size
)

validation_generator = val_gen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(height,width),
    class_mode='categorical',
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size
)

test_generator = val_gen.flow_from_directory(
    TEST_DIR,
    target_size=(height,width),
    class_mode='categorical',
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size
)

In [ ]:
#tampilkan Data Generator
import matplotlib.pyplot as plt
plt.figure(figsize = (14, 8))
for i in range(30):
    plt.subplot(5, 10, i + 1)
    
    for X, y in train_generator:

        plt.imshow(X[0])
        plt.axis("off")
        break
        
plt.tight_layout()
plt.show()

#Modelling

In [ ]:
# IMPORT SEMUA PAKET YANG ANDA PERLUKAN UNTUK MENDEFINISIKAN MODEL DISINI
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
base_model = VGG16(input_shape = (100,100,3),
                           include_top = False,
                           weights = "imagenet")
base_model.trainable = False

In [ ]:
# i add some of layer and setting dropout more big to avoid Overffiting in my model
print('Adding new layers')
x = base_model.get_layer(index = -1).output
x = Flatten()(base_model.output)
x = Dense(256,activation = "relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(64,activation = "relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x) 
print('Finished')

In [ ]:
model = Model(base_model.input, x)
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [ ]:
# setting Callback so we can save the best model in format h5 and i save it in my drive
callbacks = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')        
direc = '/content/drive/MyDrive/Kuliah/TKC/Code/Model_VGG16.h5'
best_model = ModelCheckpoint(direc, monitor='val_acc', verbose = 1, save_best_only = True)

In [ ]:
H = model.fit(train_generator,
                    epochs=100,
                    steps_per_epoch=93,  # images = batch_size * steps
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=[callbacks, best_model])

#Result

In [ ]:
import json
from tensorflow.keras.models import load_model

target_dir = '/content/drive/MyDrive/Kuliah/TKC/Code'
model_saved = load_model(target_dir + '/Model_VGG16.h5')

In [ ]:
test_score = model_saved.evaluate_generator(validation_generator, validation_generator.batch_size)
print("[INFO] accuracy: {:.2f}%".format(test_score[1] * 100)) 
print("[INFO] Loss: ",test_score[0])

In [ ]:
print(H.history.keys())

In [ ]:
import matplotlib.pyplot as plt
acc = H.history['acc']
val_acc = H.history['val_acc']
loss = H.history['loss']
val_loss = H.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()
    
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#Print Classification Report
#Print the Target names

target_names = []

for key in train_generator.class_indices:
    target_names.append(key)
print(target_names)

In [ ]:
#Classification Report
import numpy as np
from sklearn.metrics import classification_report
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
print(H.history.keys())

In [ ]:
test_datagen = ImageDataGenerator()

test_generator  = test_datagen.flow_from_directory(test_dir,
                                          class_mode=None,
                                          shuffle=False,
                                          target_size=(150, 150))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
Cmatrix_test = confusion_matrix(test_generator.classes, y_pred)
class_names = test_generator.class_indices

plt.figure(figsize = (15,12))
ax = plt.subplot()
sns.set(font_scale=2.0) # Adjust to fit
sns.heatmap(Cmatrix_test,annot=True, fmt='',ax=ax, cmap=plt.cm.Reds)

# labels, title and ticks
label_font = {'size':'15'}  # Adjust to fit
ax.set_xlabel('Predicted labels', fontdict=label_font)
ax.set_ylabel('True labels', fontdict=label_font) 

title_font = {'size':'18'}  # Adjust to fit
ax.set_title('Confusion Matrix', fontdict=title_font)

ax.tick_params(axis='both', which='major', labelsize=15)  # Adjust to fit
ax.xaxis.set_ticklabels(class_names)   
ax.yaxis.set_ticklabels(class_names)
plt.title('Confusion Matrix Test',fontsize=14)
plt.show()

#Predict

In [ ]:
import json
from tensorflow.keras.models import load_model

target_dir = '/content/drive/MyDrive/Kuliah/TKC/Code'
model_saved = load_model(target_dir + '/Model_VGG16.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import time

def pred_img(img_path):
    img = load_img(img_path, target_size=(100,100))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)

    start_pred_time = time.time()

    pred_result = model_saved.predict(img)

    end_pred_time = time.time()

    pred_value = np.argmax(pred_result[0])
    #pred_label = list(label_dict.keys())[list(label_dict.values()).index(pred_value)]
    pred_label = 'WithMask' if pred_value == 0 else 'WithoutMask' 
    confidence_percent = np.max(pred_result[0]) * 100
    pred_time = end_pred_time - start_pred_time

    return pred_label, confidence_percent, pred_time

In [ ]:
wmask_name_dir = [os.path.join(test_with_mask_dir, fname) for fname in os.listdir(test_with_mask_dir)[15:20]]
wout_name_dir = [os.path.join(test_without_mask_dir, fname) for fname in os.listdir(test_without_mask_dir)[25:30]]

wmask_wout_dir = wmask_name_dir + wout_name_dir

In [ ]:
import matplotlib.pyplot as plt
import cv2
nrows = 3
ncols = 5

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

for i, img_path in enumerate(wmask_wout_dir ):
    plt.subplot(nrows, ncols, i + 1)
    plt.subplots_adjust(hspace=1.5)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (100,100))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    true_label = img_path.split(os.path.sep)[-2]
    pred_result = pred_img(img_path)
    pred_label = pred_result[0]
    confidence_percent = pred_result[1]
    pred_time = pred_result[2]

    plt.title(f"True Label: {true_label}\nPred Label: {pred_label}\nAkurasi: {confidence_percent:.2f}%\nTime: {pred_time:.4f}s", y=1.15)
    plt.imshow(img)

plt.show()